<a href="https://colab.research.google.com/github/Kamrankhan69/Playing-With-Text/blob/main/Playing_with_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/training.1600000.processed.noemoticon.csv", encoding='latin-1',header=None)

In [ ]:
print(df)

         0           1                             2         3  \
0        0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1        0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2        0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3        0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4        0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
...     ..         ...                           ...       ...   
1599995  4  2193601966  Tue Jun 16 08:40:49 PDT 2009  NO_QUERY   
1599996  4  2193601969  Tue Jun 16 08:40:49 PDT 2009  NO_QUERY   
1599997  4  2193601991  Tue Jun 16 08:40:49 PDT 2009  NO_QUERY   
1599998  4  2193602064  Tue Jun 16 08:40:49 PDT 2009  NO_QUERY   
1599999  4  2193602129  Tue Jun 16 08:40:50 PDT 2009  NO_QUERY   

                       4                                                  5  
0        _TheSpecialOne_  @switchfoot http://twitpic.com/2y1zl - Awww, t...  
1          scotthamilton  is upset that he can't up

In [ ]:
new_columns = ["target","ids","date","flag","user","text"]

for j in range(len(df.columns)):
    old = df.columns[j]
    new = new_columns[j]
    df = df.rename(columns = {old:new})
df

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
df['target'].value_counts()

0    800000
4    800000
Name: target, dtype: int64

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords

import re
from nltk.stem import WordNetLemmatizer
stemmer = WordNetLemmatizer()
import nltk
nltk.download('omw-1.4')

def preprocess(sentence):
    tweet = re.sub(r"@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+", ' ', sentence)

    # remove all single characters
    tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', tweet)

    # Substituting multiple spaces with single space
    tweet = re.sub(r'\s+', ' ', tweet, flags=re.I)

    # Converting to Lowercase
    tweet = tweet.lower()

    # Lemmatization
    tweet = tweet.split()

    tweet = [stemmer.lemmatize(word) for word in tweet]
    tweet = ' '.join(tweet)
    return tweet



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
df['text'] = df['text'].apply(preprocess)

In [ ]:

df['target'] = df['target'].replace(4,1)
df['target']
df['text']

0          awww that a bummer you shoulda got david carr ...
1          is upset that he can update his facebook by te...
2          dived many time for the ball managed to save 5...
3               my whole body feel itchy and like it on fire
4          no it not behaving at all m mad why am here be...
                                 ...                        
1599995    just woke up having no school is the best feel...
1599996      thewdb com very cool to hear old walt interview
1599997    are you ready for your mojo makeover ask me fo...
1599998    happy 38th birthday to my boo of alll time tup...
1599999    happy charitytuesday thenspcc sparkscharity sp...
Name: text, Length: 1600000, dtype: object

In [ ]:
from keras_preprocessing.sequence import pad_sequences
max_features = 10000
maxlen = 140
X = df.iloc[:,5]
y = df.iloc[:,0]
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.3,stratify=y,random_state=1)


In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
y_train

precisionList = []
accuracyList = []
recallList = []
F1List = []





In [ ]:
import pickle
text_vectorization = layers.TextVectorization (max_tokens=10000, output_mode="int")
text_vectorization.adapt(X)
X_train = text_vectorization(X_train)
X_test = text_vectorization(X_test)

In [ ]:
import pickle
pickle.dump(X_train,open( "/content/drive/MyDrive/x_train1.p", "wb" ))
pickle.dump(X_test, open( "/content/drive/MyDrive/x_test1.p", "wb" ))
pickle.dump(y_train, open( "/content/drive/MyDrive/y_train1.p", "wb" ))
pickle.dump(y_test, open( "/content/drive/MyDrive/y_test1.p", "wb" ))


In [ ]:

X_train1 = pickle.load(open("/content/drive/MyDrive/x_train1.p","rb"))
X_test1 = pickle.load(open("/content/drive/MyDrive/x_test1.p","rb"))
y_train1 = pickle.load(open("/content/drive/MyDrive/y_train1.p","rb"))
y_test1 = pickle.load(open("/content/drive/MyDrive/y_test1.p","rb"))

# **Exprement 1 using simple RNN with 2 layers **

In [ ]:
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN
max_features = 10000
max_length = 140
model = Sequential()
model.add(Embedding(max_features, max_length))
model.add(SimpleRNN(32))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train1, y_train,
                    epochs=5,
                    batch_size=256,
                    validation_split=0.2)

Epoch 1/5
3500/3500 [==============================] - 124s 35ms/step - loss: 0.4988 - acc: 0.7585 - val_loss: 0.4415 - val_acc: 0.7962
Epoch 2/5
3500/3500 [==============================] - 127s 36ms/step - loss: 0.4383 - acc: 0.7995 - val_loss: 0.4537 - val_acc: 0.7920
Epoch 3/5
3500/3500 [==============================] - 105s 30ms/step - loss: 0.4256 - acc: 0.8059 - val_loss: 0.4331 - val_acc: 0.8016
Epoch 4/5
3500/3500 [==============================] - 102s 29ms/step - loss: 0.4172 - acc: 0.8105 - val_loss: 0.4435 - val_acc: 0.7970
Epoch 5/5
3500/3500 [==============================] - 103s 29ms/step - loss: 0.4102 - acc: 0.8148 - val_loss: 0.4290 - val_acc: 0.8050


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

model.evaluate(X_test1, y_test1)
y_predict = model.predict(X_test)

print(accuracy_score(y_test, y_predict.round()))
print(precision_score(y_test, y_predict.round()))
print(recall_score(y_test, y_predict.round()))
print(f1_score(y_test, y_predict.round()))




15000/15000 [==============================] - 75s 5ms/step
0.7790104166666667
0.8688641984829539
0.6572125
0.7483613538076136


NameError: ignored

In [ ]:
accuracyList.append(accuracy_score(y_test, y_predict.round()))
precisionList.append(precision_score(y_test, y_predict.round()))
recallList.append(recall_score(y_test, y_predict.round()))
F1List.append(f1_score(y_test, y_predict.round()))


In [ ]:
# accuracyList.append(0.7790104166666667)
# precisionList.append(0.8688641984829539)
# recallList.append(0.6572125)
# F1List.append(0.7483613538076136)


# **Exprement 2 using simple RNN with 3 layers **

In [ ]:
max_features = 10000
max_length = 140
model = Sequential()
model.add(Embedding(max_features, max_length))
model.add(SimpleRNN(32))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train1, y_train,
                    epochs=5,
                    batch_size=512,
                    validation_split=0.2)

Epoch 1/5
1750/1750 [==============================] - 69s 39ms/step - loss: 0.4898 - acc: 0.7703 - val_loss: 0.4628 - val_acc: 0.7900
Epoch 2/5
1750/1750 [==============================] - 62s 35ms/step - loss: 0.4395 - acc: 0.7984 - val_loss: 0.4348 - val_acc: 0.7979
Epoch 3/5
1750/1750 [==============================] - 64s 36ms/step - loss: 0.4248 - acc: 0.8058 - val_loss: 0.4260 - val_acc: 0.8059
Epoch 4/5
1750/1750 [==============================] - 62s 35ms/step - loss: 0.4153 - acc: 0.8114 - val_loss: 0.4318 - val_acc: 0.8031
Epoch 5/5
1750/1750 [==============================] - 63s 36ms/step - loss: 0.4080 - acc: 0.8152 - val_loss: 0.4217 - val_acc: 0.8078


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

model.evaluate(X_test1, y_test1)
y_predict = model.predict(X_test)

print(accuracy_score(y_test, y_predict.round()))
print(precision_score(y_test, y_predict.round()))
print(recall_score(y_test, y_predict.round()))
print(f1_score(y_test, y_predict.round()))


15000/15000 [==============================] - 77s 5ms/step
0.797075
0.762969984435281
0.8619208333333334
0.8094325078356412


In [ ]:
accuracyList.append(accuracy_score(y_test, y_predict.round()))
precisionList.append(precision_score(y_test, y_predict.round()))
recallList.append(recall_score(y_test, y_predict.round()))
F1List.append(f1_score(y_test, y_predict.round()))

In [ ]:
# accuracyList.append(0.797075)
# precisionList.append(0.762969984435281)
# recallList.append(0.8619208333333334)
# F1List.append(0.8094325078356412)


# **Exprement 3 using simple RNN with 2 layers using pre compiled GLove word emmbadding  **

In [ ]:
import numpy as np
from keras.models import Sequential
import keras
from keras.layers import Dense
from keras.layers import Embedding, SimpleRNN
path_to_glove_file = "/content/drive/MyDrive/glove.6B.100d.txt"
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((10000, embedding_dim))
for word, i in word_index.items():
    if i <10000:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

model = Sequential()
model.add(Embedding(max_features, 100,embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,))

model.add(SimpleRNN(32))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()
history = model.fit(X_train1, y_train1,
                    epochs=5,
                    batch_size=512,
                    validation_split=0.2)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 100)         1000000   
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 32)                4256      
                                                                 
 dense_10 (Dense)            (None, 64)                2112      
                                                                 
 dense_11 (Dense)            (None, 32)                2080      
                                                                 
 dense_12 (Dense)            (None, 1)                 33        
                                                                 
Total params: 1,008,481
Trainable params: 8,481
Non-trainable params: 1,000,000
_________________________________________________________________
Epoch 1/5
1750/1750 [===================

In [ ]:

model.evaluate(X_test1, y_test1)
y_predict = model.predict(X_test)

print(accuracy_score(y_test, y_predict.round()))
print(precision_score(y_test, y_predict.round()))
print(recall_score(y_test, y_predict.round()))
print(f1_score(y_test, y_predict.round()))


15000/15000 [==============================] - 75s 5ms/step
0.7596333333333334
0.7383743171489342
0.804225
0.7698941372625666


In [ ]:
accuracyList.append(accuracy_score(y_test, y_predict.round()))
precisionList.append(precision_score(y_test, y_predict.round()))
recallList.append(recall_score(y_test, y_predict.round()))
F1List.append(f1_score(y_test, y_predict.round()))

In [ ]:
# accuracyList.append(0.7596333333333334)
# precisionList.append(0.7383743171489342)
# recallList.append(0.804225)
# F1List.append(0.7698941372625666)

# **Exprement 4 using simple RNN with 3 layers using pre compiled GLove word emmbadding **

In [ ]:
model = Sequential()
model.add(Embedding(max_features, 100,embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,))

model.add(SimpleRNN(32))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()
history = model.fit(X_train1, y_train1,
                    epochs=5,
                    batch_size=512,
                    validation_split=0.2)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 100)         1000000   
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 32)                4256      
                                                                 
 dense_13 (Dense)            (None, 128)               4224      
                                                                 
 dense_14 (Dense)            (None, 64)                8256      
                                                                 
 dense_15 (Dense)            (None, 32)                2080      
                                                                 
 dense_16 (Dense)            (None, 1)                 33        
                                                                 
Total params: 1,018,849
Trainable params: 18,849
Non-t

In [ ]:
model.evaluate(X_test1, y_test1)
y_predict = model.predict(X_test)
print(accuracy_score(y_test, y_predict.round()))
print(precision_score(y_test, y_predict.round()))
print(recall_score(y_test, y_predict.round()))
print(f1_score(y_test, y_predict.round()))
accuracyList.append(accuracy_score(y_test, y_predict.round()))
precisionList.append(precision_score(y_test, y_predict.round()))
recallList.append(recall_score(y_test, y_predict.round()))
F1List.append(f1_score(y_test, y_predict.round()))

15000/15000 [==============================] - 77s 5ms/step
0.7596
0.7585056593753112
0.7617166666666667
0.7601077718828481


In [ ]:
# accuracyList.append(0.7596)
# precisionList.append(0.7585056593753112)
# recallList.append(0.7617166666666667)
# F1List.append(0.7601077718828481)

# **Exprement 5 using simple RNN with 2 layers using pre compiled GLove-twitter word emmbadding**

In [ ]:
import numpy as np
from keras.models import Sequential
import keras
from keras.layers import Dense
from keras.layers import Embedding, SimpleRNN
path_to_glove_file = "/content/drive/MyDrive/glove.twitter.27B.100d.txt"
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((10000, embedding_dim))
for word, i in word_index.items():
    if i <10000:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

model = Sequential()
model.add(Embedding(max_features, 100,embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,))

model.add(SimpleRNN(32))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()
history = model.fit(X_train1, y_train1,
                    epochs=5,
                    batch_size=512,
                    validation_split=0.2)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, None, 100)         1000000   
                                                                 
 simple_rnn_5 (SimpleRNN)    (None, 32)                4256      
                                                                 
 dense_17 (Dense)            (None, 64)                2112      
                                                                 
 dense_18 (Dense)            (None, 32)                2080      
                                                                 
 dense_19 (Dense)            (None, 1)                 33        
                                                                 
Total params: 1,008,481
Trainable params: 8,481
Non-trainable params: 1,000,000
_________________________________________________________________
Epoch 1/5
1750/1750 [===================

In [ ]:
model.evaluate(X_test1, y_test1)
y_predict = model.predict(X_test)
print(accuracy_score(y_test, y_predict.round()))
print(precision_score(y_test, y_predict.round()))
print(recall_score(y_test, y_predict.round()))
print(f1_score(y_test, y_predict.round()))
accuracyList.append(accuracy_score(y_test, y_predict.round()))
precisionList.append(precision_score(y_test, y_predict.round()))
recallList.append(recall_score(y_test, y_predict.round()))
F1List.append(f1_score(y_test, y_predict.round()))

15000/15000 [==============================] - 75s 5ms/step
0.78329375
0.7892215874528354
0.7730458333333333
0.7810499683211914


In [ ]:
# accuracyList.append(0.78329375)
# precisionList.append(0.7892215874528354)
# recallList.append(0.7730458333333333)
# F1List.append(0.7810499683211914)

# **Exprement 6 using simple RNN with 3 layers using pre compiled GLove-twitter word emmbadding **

In [ ]:
model = Sequential()
model.add(Embedding(max_features, 100,embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,))

model.add(SimpleRNN(32))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()
history = model.fit(X_train1, y_train1,
                    epochs=5,
                    batch_size=512,
                    validation_split=0.2)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, None, 100)         1000000   
                                                                 
 simple_rnn_6 (SimpleRNN)    (None, 32)                4256      
                                                                 
 dense_20 (Dense)            (None, 128)               4224      
                                                                 
 dense_21 (Dense)            (None, 64)                8256      
                                                                 
 dense_22 (Dense)            (None, 32)                2080      
                                                                 
 dense_23 (Dense)            (None, 1)                 33        
                                                                 
Total params: 1,018,849
Trainable params: 18,849
Non-t

In [ ]:
model.evaluate(X_test1, y_test1)
y_predict = model.predict(X_test)
print(accuracy_score(y_test, y_predict.round()))
print(precision_score(y_test, y_predict.round()))
print(recall_score(y_test, y_predict.round()))
print(f1_score(y_test, y_predict.round()))
accuracyList.append(accuracy_score(y_test, y_predict.round()))
precisionList.append(precision_score(y_test, y_predict.round()))
recallList.append(recall_score(y_test, y_predict.round()))
F1List.append(f1_score(y_test, y_predict.round()))

15000/15000 [==============================] - 80s 5ms/step
0.7702104166666667
0.809772198577495
0.7063541666666666
0.7545360065338967


In [ ]:
# accuracyList.append(0.7702104166666667)
# precisionList.append(0.809772198577495)
# recallList.append(0.7063541666666666)
# F1List.append(0.7545360065338967)

# **Exprement 7 using LSTM with 2 layers **

In [ ]:
from tensorflow.keras.layers import LSTM
from keras.models import Sequential
max_features = 10000
max_length = 140
model = Sequential()
model.add(Embedding(max_features, max_length))
model.add(LSTM(32))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train1, y_train,
                    epochs=5,
                    batch_size=256,
                    validation_split=0.2)

Epoch 1/5
3500/3500 [==============================] - 34s 8ms/step - loss: 0.4498 - acc: 0.7886 - val_loss: 0.4201 - val_acc: 0.8095
Epoch 2/5
3500/3500 [==============================] - 30s 9ms/step - loss: 0.4088 - acc: 0.8133 - val_loss: 0.4083 - val_acc: 0.8144
Epoch 3/5
3500/3500 [==============================] - 28s 8ms/step - loss: 0.3949 - acc: 0.8212 - val_loss: 0.3996 - val_acc: 0.8184
Epoch 4/5
3500/3500 [==============================] - 29s 8ms/step - loss: 0.3846 - acc: 0.8266 - val_loss: 0.3945 - val_acc: 0.8205
Epoch 5/5
3500/3500 [==============================] - 31s 9ms/step - loss: 0.3766 - acc: 0.8308 - val_loss: 0.3904 - val_acc: 0.8233


In [ ]:
model.evaluate(X_test1, y_test1)
y_predict = model.predict(X_test)
print(accuracy_score(y_test, y_predict.round()))
print(precision_score(y_test, y_predict.round()))
print(recall_score(y_test, y_predict.round()))
print(f1_score(y_test, y_predict.round()))
accuracyList.append(accuracy_score(y_test, y_predict.round()))
precisionList.append(precision_score(y_test, y_predict.round()))
recallList.append(recall_score(y_test, y_predict.round()))
F1List.append(f1_score(y_test, y_predict.round()))

15000/15000 [==============================] - 43s 3ms/step
0.8185145833333334
0.7897683551357232
0.8681166666666666
0.8270912190929498


In [ ]:
# accuracyList.append(0.8185145833333334)
# precisionList.append(0.7897683551357232)
# recallList.append(0.8681166666666666)
# F1List.append(0.8270912190929498)

# *Exprement 8 using LSTM with 3 layers*

In [ ]:
max_features = 10000
max_length = 140
model = Sequential()
model.add(Embedding(max_features, max_length))
model.add(LSTM(32))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train1, y_train,
                    epochs=5,
                    batch_size=256,
                    validation_split=0.2)

Epoch 1/5
3500/3500 [==============================] - 30s 8ms/step - loss: 0.4561 - acc: 0.7831 - val_loss: 0.4179 - val_acc: 0.8092
Epoch 2/5
3500/3500 [==============================] - 30s 9ms/step - loss: 0.4088 - acc: 0.8139 - val_loss: 0.4038 - val_acc: 0.8154
Epoch 3/5
3500/3500 [==============================] - 28s 8ms/step - loss: 0.3941 - acc: 0.8218 - val_loss: 0.4102 - val_acc: 0.8167
Epoch 4/5
3500/3500 [==============================] - 28s 8ms/step - loss: 0.3836 - acc: 0.8275 - val_loss: 0.3941 - val_acc: 0.8223
Epoch 5/5
3500/3500 [==============================] - 28s 8ms/step - loss: 0.3756 - acc: 0.8315 - val_loss: 0.3913 - val_acc: 0.8241


In [ ]:
model.evaluate(X_test1, y_test1)
y_predict = model.predict(X_test)
print(accuracy_score(y_test, y_predict.round()))
print(precision_score(y_test, y_predict.round()))
print(recall_score(y_test, y_predict.round()))
print(f1_score(y_test, y_predict.round()))
accuracyList.append(accuracy_score(y_test, y_predict.round()))
precisionList.append(precision_score(y_test, y_predict.round()))
recallList.append(recall_score(y_test, y_predict.round()))
F1List.append(f1_score(y_test, y_predict.round()))

15000/15000 [==============================] - 42s 3ms/step
0.8233625
0.8126490589145288
0.8404958333333333
0.8263379104673265


In [ ]:
# accuracyList.append(0.8233625)
# precisionList.append(0.8126490589145288)
# recallList.append(0.8404958333333333)
# F1List.append(0.8263379104673265)

# *Exprement 9 using LSTM with 2 layers using pre compiled GLove word
#  emmbadding*

In [ ]:
path_to_glove_file = "/content/drive/MyDrive/glove.6B.100d.txt"
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((10000, embedding_dim))
for word, i in word_index.items():
    if i <10000:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

model = Sequential()
model.add(Embedding(max_features, 100,embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,))

model.add(LSTM(32))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()
history = model.fit(X_train1, y_train1,
                    epochs=5,
                    batch_size=512,
                    validation_split=0.2)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, None, 100)         1000000   
                                                                 
 lstm_2 (LSTM)               (None, 32)                17024     
                                                                 
 dense_31 (Dense)            (None, 64)                2112      
                                                                 
 dense_32 (Dense)            (None, 32)                2080      
                                                                 
 dense_33 (Dense)            (None, 1)                 33        
                                                                 
Total params: 1,021,249
Trainable params: 21,249
Non-trainable params: 1,000,000
_________________________________________________________________
Epoch 1/5
1750/1750 [==================

In [ ]:
model.evaluate(X_test1, y_test1)
y_predict = model.predict(X_test)
print(accuracy_score(y_test, y_predict.round()))
print(precision_score(y_test, y_predict.round()))
print(recall_score(y_test, y_predict.round()))
print(f1_score(y_test, y_predict.round()))
accuracyList.append(accuracy_score(y_test, y_predict.round()))
precisionList.append(precision_score(y_test, y_predict.round()))
recallList.append(recall_score(y_test, y_predict.round()))
F1List.append(f1_score(y_test, y_predict.round()))

15000/15000 [==============================] - 48s 3ms/step
0.7948166666666666
0.7731470043236566
0.8344833333333334
0.8026450785508176


In [ ]:
# accuracyList.append(0.7948166666666666)
# precisionList.append(0.7731470043236566)
# recallList.append(0.8344833333333334)
# F1List.append(0.8026450785508176)

# **Exprement 10 using LSTM with 3 layers using pre compiled GLove word emmbadding **

In [ ]:
model = Sequential()
model.add(Embedding(max_features, 100,embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,))

model.add(LSTM(32))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()
history = model.fit(X_train1, y_train1,
                    epochs=5,
                    batch_size=512,
                    validation_split=0.2)

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, None, 100)         1000000   
                                                                 
 lstm_3 (LSTM)               (None, 32)                17024     
                                                                 
 dense_34 (Dense)            (None, 128)               4224      
                                                                 
 dense_35 (Dense)            (None, 64)                8256      
                                                                 
 dense_36 (Dense)            (None, 32)                2080      
                                                                 
 dense_37 (Dense)            (None, 1)                 33        
                                                                 
Total params: 1,031,617
Trainable params: 31,617
Non-

In [ ]:
model.evaluate(X_test1, y_test1)
y_predict = model.predict(X_test)
print(accuracy_score(y_test, y_predict.round()))
print(precision_score(y_test, y_predict.round()))
print(recall_score(y_test, y_predict.round()))
print(f1_score(y_test, y_predict.round()))
accuracyList.append(accuracy_score(y_test, y_predict.round()))
precisionList.append(precision_score(y_test, y_predict.round()))
recallList.append(recall_score(y_test, y_predict.round()))
F1List.append(f1_score(y_test, y_predict.round()))

15000/15000 [==============================] - 47s 3ms/step
0.7868770833333333
0.7488371420620263
0.8633125
0.802010493739948


In [ ]:
# accuracyList.append(0.7868770833333333)
# precisionList.append(0.7488371420620263)
# recallList.append(0.8633125)
# F1List.append(0.802010493739948)

# **Exprement 11 using LSTM with 2 layers using pre compiled GLove-twitter word emmbadding **

In [ ]:
path_to_glove_file = "/content/drive/MyDrive/glove.twitter.27B.100d.txt"
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((10000, embedding_dim))
for word, i in word_index.items():
    if i <10000:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

model = Sequential()
model.add(Embedding(max_features, 100,embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,))

model.add(LSTM(32))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()
history = model.fit(X_train1, y_train1,
                    epochs=5,
                    batch_size=512,
                    validation_split=0.2)

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, None, 100)         1000000   
                                                                 
 lstm_4 (LSTM)               (None, 32)                17024     
                                                                 
 dense_38 (Dense)            (None, 64)                2112      
                                                                 
 dense_39 (Dense)            (None, 32)                2080      
                                                                 
 dense_40 (Dense)            (None, 1)                 33        
                                                                 
Total params: 1,021,249
Trainable params: 21,249
Non-trainable params: 1,000,000
_________________________________________________________________
Epoch 1/5
1750/1750 [=================

In [ ]:
model.evaluate(X_test1, y_test1)
y_predict = model.predict(X_test)
print(accuracy_score(y_test, y_predict.round()))
print(precision_score(y_test, y_predict.round()))
print(recall_score(y_test, y_predict.round()))
print(f1_score(y_test, y_predict.round()))
accuracyList.append(accuracy_score(y_test, y_predict.round()))
precisionList.append(precision_score(y_test, y_predict.round()))
recallList.append(recall_score(y_test, y_predict.round()))
F1List.append(f1_score(y_test, y_predict.round()))

In [ ]:
# accuracyList.append(0.7968770833333333)
# precisionList.append(0.7588371420620263)
# recallList.append(0.7633125)
# F1List.append(0.802010493739948)

# **Exprement 12 using LSTM with 3 layers using pre compiled GLove-twitter word emmbadding **

In [ ]:
model = Sequential()
model.add(Embedding(max_features, 100,embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,))

model.add(LSTM(32))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()
history = model.fit(X_train1, y_train1,
                    epochs=5,
                    batch_size=512,
                    validation_split=0.2)

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, None, 100)         1000000   
                                                                 
 lstm_5 (LSTM)               (None, 32)                17024     
                                                                 
 dense_41 (Dense)            (None, 128)               4224      
                                                                 
 dense_42 (Dense)            (None, 64)                8256      
                                                                 
 dense_43 (Dense)            (None, 32)                2080      
                                                                 
 dense_44 (Dense)            (None, 1)                 33        
                                                                 
Total params: 1,031,617
Trainable params: 31,617
Non-

In [ ]:
model.evaluate(X_test1, y_test1)
y_predict = model.predict(X_test)
print(accuracy_score(y_test, y_predict.round()))
print(precision_score(y_test, y_predict.round()))
print(recall_score(y_test, y_predict.round()))
print(f1_score(y_test, y_predict.round()))
accuracyList.append(accuracy_score(y_test, y_predict.round()))
precisionList.append(precision_score(y_test, y_predict.round()))
recallList.append(recall_score(y_test, y_predict.round()))
F1List.append(f1_score(y_test, y_predict.round()))

15000/15000 [==============================] - 38s 3ms/step
0.8059125
0.830582196406862
0.7686
0.7983899240407712


In [ ]:
# accuracyList.append(0.8059125)
# precisionList.append(0.830582196406862)
# recallList.append(0.7686)
# F1List.append(0.7983899240407712)

In [ ]:
pickle.dump(accuracyList,open("/content/drive/MyDrive/acc_list.p","wb"))
pickle.dump(precisionList,open("/content/drive/MyDrive/pre_list.p","wb"))
pickle.dump(recallList,open("/content/drive/MyDrive/recall_list.p","wb"))
pickle.dump(F1List,open("/content/drive/MyDrive/F1_list.p","wb"))


In [ ]:
acc_list = pickle.load(open("/content/drive/MyDrive/acc_list.p","rb"))
pre_list = pickle.load(open("/content/drive/MyDrive/pre_list.p","rb"))
recall_list = pickle.load(open("/content/drive/MyDrive/recall_list.p","rb"))
F1_list = pickle.load(open("/content/drive/MyDrive/F1_list.p","rb"))

In [ ]:
print(len(acc_list))
print(len(pre_list))
print(len(recall_list))
print(len(F1_list))


12
12
12
12


# **Expriment 13 using simple RNN with 2 layers using pre compiled FastText word emmbadding **

In [ ]:
import numpy as np
from keras.models import Sequential
import keras
from keras.layers import Dense
from keras.layers import Embedding, SimpleRNN
path_to_glove_file = "/content/drive/MyDrive/FastText(wiki-news-300d-1M).vec"
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

embedding_dim = 300

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((10000, embedding_dim))
for word, i in word_index.items():
    if i <10000:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

model = Sequential()
model.add(Embedding(max_features, 300,embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,))

model.add(SimpleRNN(32))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()
history = model.fit(X_train1, y_train1,
                    epochs=5,
                    batch_size=512,
                    validation_split=0.2)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 300)         3000000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                10656     
                                                                 
 dense (Dense)               (None, 64)                2112      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3,014,881
Trainable params: 14,881
Non-trainable params: 3,000,000
_________________________________________________________________
Epoch 1/5
1750/1750 [==================

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
model.evaluate(X_test1, y_test1)
y_predict = model.predict(X_test)
print(accuracy_score(y_test, y_predict.round()))
print(precision_score(y_test, y_predict.round()))
print(recall_score(y_test, y_predict.round()))
print(f1_score(y_test, y_predict.round()))
acc_list.append(accuracy_score(y_test, y_predict.round()))
pre_list.append(precision_score(y_test, y_predict.round()))
recall_list.append(recall_score(y_test, y_predict.round()))
F1_list.append(f1_score(y_test, y_predict.round()))

15000/15000 [==============================] - 134s 9ms/step
0.758625
0.8191719116376651
0.663775
0.7333314920178977


# **Expriment 14 using simple RNN with 3 layers using pre compiled FastText word emmbadding **

In [ ]:
model = Sequential()
model.add(Embedding(max_features, 300,embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,))

model.add(SimpleRNN(32))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()
history = model.fit(X_train1, y_train1,
                    epochs=5,
                    batch_size=512,
                    validation_split=0.2)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 300)         3000000   
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 32)                10656     
                                                                 
 dense_3 (Dense)             (None, 128)               4224      
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3,025,249
Trainable params: 25,249
Non-t

In [ ]:
model.evaluate(X_test1, y_test1)
y_predict = model.predict(X_test)
print(accuracy_score(y_test, y_predict.round()))
print(precision_score(y_test, y_predict.round()))
print(recall_score(y_test, y_predict.round()))
print(f1_score(y_test, y_predict.round()))
acc_list.append(accuracy_score(y_test, y_predict.round()))
pre_list.append(precision_score(y_test, y_predict.round()))
recall_list.append(recall_score(y_test, y_predict.round()))
F1_list.append(f1_score(y_test, y_predict.round()))

15000/15000 [==============================] - 77s 5ms/step
0.77120625
0.7516829906773337
0.8099916666666667
0.7797487851396965


# **Expriment 15 using LSTM with 2 layers using pre compiled FastText word emmbadding **

In [ ]:
import numpy as np
from keras.models import Sequential
from tensorflow.keras.layers import LSTM
import keras
from keras.layers import Dense
from keras.layers import Embedding, SimpleRNN
path_to_glove_file = "/content/drive/MyDrive/FastText(wiki-news-300d-1M).vec"
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

embedding_dim = 300

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((10000, embedding_dim))
for word, i in word_index.items():
    if i <10000:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

model = Sequential()
model.add(Embedding(max_features, 300,embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,))

model.add(LSTM(32))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()
history = model.fit(X_train1, y_train1,
                    epochs=5,
                    batch_size=512,
                    validation_split=0.2)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, None, 300)         3000000   
                                                                 
 lstm (LSTM)                 (None, 32)                42624     
                                                                 
 dense_7 (Dense)             (None, 64)                2112      
                                                                 
 dense_8 (Dense)             (None, 32)                2080      
                                                                 
 dense_9 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3,046,849
Trainable params: 46,849
Non-trainable params: 3,000,000
_________________________________________________________________
Epoch 1/5
1750/1750 [==================

In [ ]:
model.evaluate(X_test1, y_test1)
y_predict = model.predict(X_test)
print(accuracy_score(y_test, y_predict.round()))
print(precision_score(y_test, y_predict.round()))
print(recall_score(y_test, y_predict.round()))
print(f1_score(y_test, y_predict.round()))
acc_list.append(accuracy_score(y_test, y_predict.round()))
pre_list.append(precision_score(y_test, y_predict.round()))
recall_list.append(recall_score(y_test, y_predict.round()))
F1_list.append(f1_score(y_test, y_predict.round()))

15000/15000 [==============================] - 52s 3ms/step
0.8118
0.8251303441084463
0.7913
0.8078611536498213


# **Expriment 16 using LSTM with 2 layers using pre compiled FastText word emmbadding **

In [ ]:
model = Sequential()
model.add(Embedding(max_features, 300,embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,))

model.add(LSTM(32))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()
history = model.fit(X_train1, y_train1,
                    epochs=5,
                    batch_size=512,
                    validation_split=0.2)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, None, 300)         3000000   
                                                                 
 lstm_1 (LSTM)               (None, 32)                42624     
                                                                 
 dense_10 (Dense)            (None, 128)               4224      
                                                                 
 dense_11 (Dense)            (None, 64)                8256      
                                                                 
 dense_12 (Dense)            (None, 32)                2080      
                                                                 
 dense_13 (Dense)            (None, 1)                 33        
                                                                 
Total params: 3,057,217
Trainable params: 57,217
Non-t

In [ ]:
model.evaluate(X_test1, y_test1)
y_predict = model.predict(X_test)
print(accuracy_score(y_test, y_predict.round()))
print(precision_score(y_test, y_predict.round()))
print(recall_score(y_test, y_predict.round()))
print(f1_score(y_test, y_predict.round()))
acc_list.append(accuracy_score(y_test, y_predict.round()))
pre_list.append(precision_score(y_test, y_predict.round()))
recall_list.append(recall_score(y_test, y_predict.round()))
F1_list.append(f1_score(y_test, y_predict.round()))

15000/15000 [==============================] - 43s 3ms/step
0.8109125
0.7997469269703543
0.8295375
0.8143698613326789


In [ ]:
import pickle
acc_list = pickle.load(open("/content/drive/MyDrive/acc_list.p","rb"))
pre_list = pickle.load(open("/content/drive/MyDrive/pre_list.p","rb"))
recall_list = pickle.load(open("/content/drive/MyDrive/recall_list.p","rb"))
F1_list = pickle.load(open("/content/drive/MyDrive/F1_list.p","rb"))


In [ ]:
pickle.dump(acc_list,open("/content/drive/MyDrive/acc_list.p","wb"))
pickle.dump(pre_list,open("/content/drive/MyDrive/pre_list.p","wb"))
pickle.dump(recall_list,open("/content/drive/MyDrive/recall_list.p","wb"))
pickle.dump(F1_list,open("/content/drive/MyDrive/F1_list.p","wb"))


In [ ]:
acc_list1 = acc_list
pre_list1 = pre_list
recall_list1 = recall_list
F1_list1 = F1_list
for i in range(len(acc_list)):
  acc_list1[i] = "Expriment "+str(i+1)+ " "+str(acc_list1[i])
  pre_list1[i] = "Expriment "+str(i+1)+ " "+str(pre_list1[i])
  recall_list1[i] = "Expriment "+str(i+1)+ " "+str(recall_list1[i])
  F1_list1[i] = "Expriment "+str(i+1)+ " "+str(F1_list1[i])


In [ ]:
acc_list.sort()
pre_list.sort()
recall_list.sort()
F1_list.sort()

acc_list1.sort()
pre_list1.sort()
recall_list1.sort()
F1_list1.sort()

best_accurcy = ''
best_precision = ''
best_recall = ''
best_F1_score = ''
for i in range(len(acc_list)):
  if str(acc_list[-1]) in str(acc_list1[i]):
      best_accurcy = str(acc_list1[i])
  if str(pre_list[-1]) in str(pre_list1[i]):
      best_precision = str(pre_list1[i])
  if str(recall_list[-1]) in str(recall_list1[i]):
      best_recall = str(recall_list1[i])
  if str(F1_list[-1]) in str(F1_list1[i]):
      best_F1_score = str(F1_list1[i])



In [ ]:
print(best_accurcy)
print(best_precision)
print(best_recall)
print(best_F1_score)

Expriment 8 0.8233625
Expriment 1 0.8688641984829539
Expriment 7 0.8681166666666666
Expriment 7 0.8270912190929498


# According to above 16 expriments

#Expriment 7 which simple LSTM with 2 layers is the best because it is giving the best out_put on both RECALL and F1-Score

#thats why it is best among all others